In [175]:
import string
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize, regexp_tokenize
from autocorrect import spell
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from itertools import chain
from gensim.models import KeyedVectors
import numpy as np
from numpy.random import randn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import time
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB                                                                
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import accuracy_score

In [2]:
df_train = pd.read_csv("train.csv")

In [3]:
sentence_1a = df_train['question1']
sentence_1b = df_train['question2']
is_duplicate = df_train['is_duplicate']
id_ = df_train['id']

In [4]:
model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz",binary=True)

In [54]:
STOPWORDS = stopwords.words('english')
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
punctuation_list = list(string.punctuation)

'''
def spell_digits(text):
    translation = {
        '0': 'zero',
        '1': 'one',
        '2': 'two',
        '3': 'three',
        '4': 'four',
        '5': 'five',
        '6': 'six',
        '7': 'seven',
        '8': 'eight',
        '9': 'nine',
    }
    return translate(text, translation)
'''

def spelling_correction(question):
    question_updated = ""
    #for token in word_tokenize(question):
    for token in tokenizer.tokenize(question):
        word = ''.join(ch for ch in token if ch not in punctuation_list and ch.isalpha())
        question_updated =  question_updated + spell(word) + " "
    return question_updated

def remove_stopword_questions(question):
    temp_list = [token for token in tokenizer.tokenize(question) if token not in stopwords.words('english')]
    return ' '.join(temp_list)

def get_preprocessed_tokens(question):  
    question = spelling_correction(question)
    question = question.lower()
    question = remove_stopword_questions(question)
    question = [word for word in tokenizer.tokenize(question)]
    return question

In [6]:
def lemmatize_tokens(question):
    joined_tokens = []
    for token in get_preprocessed_tokens(question):
        token_lemma = wordnet_lemmatizer.lemmatize(token)
        joined_tokens.append(token_lemma)
    return joined_tokens

In [ ]:
df_word2vec = pd.DataFrame()
for c_id, q1 , q2, label in zip(id_, sentence_1a, sentence_1b, is_duplicate):
    questions = [q1, q2]
    try:
        lemmas = []
        for question in questions:
            lemmas.append(lemmatize_tokens(question))

        embeddings = []
        for lemma in lemmas:
            vec_word = np.zeros(300)
            no_of_words = 0.0
            for word in lemma:
                #if word in model.vocab:
                vec_word += model.word_vec(word)
                no_of_words += 1
            '''
            if no_of_words == 0:
                vec_word = np.zeros(300)
            else:
            '''
            vec_word = vec_word/no_of_words

            embeddings.append(vec_word.reshape(1,-1))

        cos = cosine_similarity(embeddings[0],embeddings[1])[0]
        df_word2vec_temp = pd.DataFrame({'embedding1': [embeddings[0][0]], 'embedding2': [embeddings[1][0]], \
                                         'cosine': cos, 'is_duplicate': label, \
                                        'question1': q1, 'question2': q2})
        df_word2vec = pd.concat([df_word2vec, df_word2vec_temp])
        
    except:
        print c_id

In [94]:
df_word2vec.tail(100)

,cosine,embedding1,embedding2,is_duplicate,question1,question2
0,0.721876,"[-0.179036458333, -0.17138671875, -0.059082031...","[-0.0510635375977, -0.0872802734375, -0.039848...",0,What is the pink drink from Starbucks?,Can I stay in Starbucks for an entire day as l...
0,0.748472,"[-0.083740234375, 0.106689453125, 0.1234537760...","[-0.0101841517857, 0.0188511439732, 0.00662667...",0,Which are the best Best Pictures in Academy Aw...,Who has won a best actor at the Academy Awards...
0,0.284414,"[0.0281982421875, 0.0219116210938, -0.02606201...","[0.0927530924479, 0.0633951822917, -0.04284667...",0,What are some of the purposes of promotional t...,What exercises should I do to lose weight at h...
0,0.822010,"[0.039111328125, 0.00260009765625, 0.124951171...","[0.0653827373798, -0.0503399188702, 0.05968299...",0,If you could ask every human presently alive o...,If you could get now instantly and completely ...
0,0.934587,"[0.0511779785156, 0.0625, -0.0662536621094, 0....","[0.00930786132812, 0.0523681640625, 0.00988769...",1,What are some good sites for learning Python?,What are the best learning sites for Python?
0,0.685158,"[-0.110961914062, -0.060546875, 0.03876953125,...","[-0.0477905273438, -0.120483398438, -0.0767822...",0,How do I make iPhone clock show seconds?,Should I buy second hand iPhone 6s for INR 15000?
0,0.924393,"[0.00203450520833, 0.064951578776, 0.103719075...","[0.010498046875, 0.0634841918945, 0.0838928222...",1,How do you start a hedge fund?,How can I start a hedge fund after college?
0,0.565857,"[0.124267578125, -0.0167643229167, -0.21468098...","[0.056982421875, 0.00028076171875, -0.00185546...",0,How much I can earn through blogging?,How can one make money starting a blog?
0,0.891367,"[-0.0788922991071, 0.0617327008929, -0.0268445...","[-0.000209263392857, 0.0182059151786, 0.011311...",0,What is the corporate culture like at Tiffany?...,What is the corporate culture like at Quantum?...
0,0.522218,"[0.082275390625, -0.012451171875, 0.0616455078...","[0.225341796875, 0.067138671875, 0.11254882812...",0,What is the most overrated pleasure? Why?,What is your most sorrowful pleasure?


In [95]:
df_word2vec.to_csv("embeddings_without_stopwords_words_deleted.csv", index=False)
df_word2vec_org = df_word2vec

In [117]:
df_word2vec_org = df_word2vec

In [118]:
df_word2vec_org

,cosine,embedding1,embedding2,is_duplicate,question1,question2
0,0.931028,"[-0.117213657924, 0.0111607142857, -0.02437046...","[-0.0976867675781, 0.0249837239583, -0.0301920...",0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
0,0.777991,"[0.0597330729167, -0.00455220540365, 0.0519612...","[0.00126953125, 0.0135009765625, -0.0549804687...",0,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
0,0.345831,"[-0.0293782552083, 0.0596516927083, -0.2041854...","[0.148217773438, -0.021923828125, 0.0549316406...",0,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
0,0.633217,"[-0.0685180664062, 0.0075439453125, 0.08830566...","[-0.05009765625, 0.14853515625, 0.038427734375...",0,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?
0,0.935385,"[-0.067626953125, -0.1103515625, -0.0213623046...","[-0.052001953125, -0.03759765625, -0.026855468...",1,How can I be a good geologist?,What should I do to be a great geologist?
0,1.000000,"[0.155029296875, -0.0684204101562, 0.144775390...","[0.155029296875, -0.0684204101562, 0.144775390...",0,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?"
0,0.907822,"[-0.0890808105469, -0.0761413574219, -0.057983...","[-0.124430338542, -0.04736328125, -0.013183593...",1,How do I read and find my YouTube comments?,How can I see all my Youtube comments?
0,1.000000,"[0.0268707275391, 0.0444030761719, -0.03454589...","[0.0268707275391, 0.0444030761719, -0.03454589...",1,What can make Physics easy to learn?,How can you make physics easy to learn?
0,0.945967,"[0.0632781982422, 0.0108642578125, 0.006732940...","[0.0498657226562, -0.0314127604167, 0.00996907...",1,What was your first sexual experience like?,What was your first sexual experience?
0,0.970117,"[-0.120119730632, 0.0572408040365, 0.093526204...","[-0.109052022298, 0.076904296875, 0.1361694335...",0,What are the laws to change your status from a...,What are the laws to change your status from a...


In [126]:
threshold_list = [0.5 , 0.6 , 0.7, 0.8, 0.9]
cosine = df_word2vec_org['cosine']
y_true = df_word2vec_org['is_duplicate']
for threshold in threshold_list:
    is_dup_pred = []
    for cos in cosine:
        if cos > threshold:
            is_dup_pred.append(1)
        else:
            is_dup_pred.append(0)
        
    df_word2vec_org['is_dup_pred'+ str(threshold)] = is_dup_pred
    y_pred = is_dup_pred
    target_names = [0, 1]
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=target_names).ravel()
    print confusion_matrix(y_true, y_pred, labels=target_names)
    accuracy = float(tn + tp)/(tn + fp + fn + tp)
    print "Accuracy: ", accuracy
    print classification_report(y_true, y_pred)

[[ 32072 172294]
 [   647 134277]]
Accuracy:  0.490285596392
             precision    recall  f1-score   support

          0       0.98      0.16      0.27    204366
          1       0.44      1.00      0.61    134924

avg / total       0.76      0.49      0.40    339290

[[ 50450 153916]
 [  2905 132019]]
Accuracy:  0.537796575201
             precision    recall  f1-score   support

          0       0.95      0.25      0.39    204366
          1       0.46      0.98      0.63    134924

avg / total       0.75      0.54      0.49    339290

[[ 83996 120370]
 [ 11982 122942]]
Accuracy:  0.609914822129
             precision    recall  f1-score   support

          0       0.88      0.41      0.56    204366
          1       0.51      0.91      0.65    134924

avg / total       0.73      0.61      0.60    339290

[[127072  77294]
 [ 37859  97065]]
Accuracy:  0.660605971293
             precision    recall  f1-score   support

          0       0.77      0.62      0.69    204366
    

In [121]:
df_word2vec_org.shape

(339290, 11)

In [127]:
df_word2vec_org.to_csv("embeddings_without_stopwords_words_deleted.csv", index=False)

In [184]:
embedding1 = df_word2vec_org['embedding1']
embedding2 = df_word2vec_org['embedding2']
train_label = df_word2vec_org['is_duplicate']
train_questions = embedding1 + embedding2

In [177]:
type(embedding1[0:5]), embedding2[0:5]

(pandas.core.series.Series,
 0    [-0.0976867675781, 0.0249837239583, -0.0301920...
 0    [0.00126953125, 0.0135009765625, -0.0549804687...
 0    [0.148217773438, -0.021923828125, 0.0549316406...
 0    [-0.05009765625, 0.14853515625, 0.038427734375...
 0    [-0.052001953125, -0.03759765625, -0.026855468...
 Name: embedding2, dtype: object)

In [139]:
train_questions

0    [-0.214900425502, 0.036144438244, -0.054562523...
0    [0.0610026041667, 0.00894877115885, -0.0030192...
0    [0.118839518229, 0.0377278645833, -0.149253845...
0    [-0.118615722656, 0.156079101562, 0.1267333984...
0    [-0.11962890625, -0.14794921875, -0.0482177734...
dtype: object

In [182]:
def naive_bayer(x_train_matrix, x_test_matrix, y_train, y_test):    #NAIVE BAYERS MODEL
                                                                    
    nb = GaussianNB()
    nb.fit(x_train_matrix, y_train)                                 #train the model
    y_pred = nb.predict(x_test_matrix)                              #make predictions for X_test
    print ("Naive Byers Model score: " + str(accuracy_score(y_test, y_pred)))         
    print ("Naive Byers Model confusion matrix:")
    print (confusion_matrix(y_test, y_pred))

def logistic_regression(x_train_matrix, x_test_matrix, y_train, y_test): 
    logReg = LogisticRegression(C=0.85)
    logReg.fit(x_train_matrix, y_train)
    y_pred_log = logReg.predict(x_test_matrix)
    print ("Logistic Regression score: "+ str(accuracy_score(y_test, y_pred_log)))
    print ("Logistic Regression confusion matrix:")
    print (confusion_matrix(y_test, y_pred_log))

def linear_svm(x_train_matrix, x_test_matrix, y_train, y_test):
    svm1 = LinearSVC(C=1)
    svm1.fit(x_train_matrix, y_train)
    y_pred_svc = svm1.predict(x_test_matrix)
    print ("SVC score: " + str(accuracy_score(y_test, y_pred_svc)))
    print ("SVC confusionmatrix:")
    print (confusion_matrix(y_test, y_pred_svc))

In [ ]:
start_time = time.time()


train_question_label = zip(train_questions,train_label) 
train_df = pd.DataFrame(train_question_label, columns=['embedding', 'label'])
X = train_df.embedding
Y = train_df.label

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=1)

preprocessing_time = time.time()
print("--- start -> preprocessing:%s seconds ---" % (preprocessing_time - start_time))

X_train = np.matrix(X_train.tolist())
#Y_train = np.matrix(Y_train.tolist())
X_test = np.matrix(X_test.tolist())
#Y_test = np.matrix(Y_test.tolist())
naive_bayer(X_train, X_test, Y_train, Y_test)
naive_bayer_time = time.time() 
print("--- preprocessing -> naive bayer :%s seconds ---" % (naive_bayer_time - preprocessing_time))

print ("-------------------------------------------")

logistic_regression(X_train, X_test, Y_train, Y_test)
logistic_regression_time = time.time()
print("--- niave bayer -> logistic regression :%s seconds ---" % (logistic_regression_time - naive_bayer_time))

print ("-------------------------------------------")

linear_svm(X_train, X_test, Y_train, Y_test)
linear_svm_time = time.time()
print("--- logistic regression -> linear svm :%s seconds ---" % (linear_svm_time - logistic_regression_time))
print("--- TOTAL TIME: %s seconds ---" % (time.time() - start_time))

--- start -> preprocessing:0.460880994797 seconds ---
Naive Byers Model score: 0.618499699374
Naive Byers Model confusion matrix:
[[36178 14946]
 [17414 16285]]
--- preprocessing -> naive bayer :5.19384503365 seconds ---
-------------------------------------------
Logistic Regression score: 0.677917545949
Logistic Regression confusion matrix:
[[43475  7649]
 [19671 14028]]
--- niave bayer -> logistic regression :17.6321380138 seconds ---
-------------------------------------------
